In [1]:
import sys
sys.path.append('./')
import numpy as np
import pandas as pd
import neural_tape_controller
#Положительные числа - положительные награды.
import tasks 
import optimize
import pickle
root_dir = './'

In [2]:
nt = neural_tape_controller.nt_controller(tacts=2,input_size=36,output_size=5)
genom = nt.nn.disassemble_genom()

In [ ]:
try:
    with open(root_dir+f'./genoms/best_genoms_aa_gun2.pkl', 'rb') as f:
        genom = pickle.load(f)[-1]
    print('loaded successfully')
except Exception:
    pass

In [3]:
%%time
q=tasks.custom_test(genom, verbose=True, draw=False)
print('q=',q)

aa_gun_2: [-180 -150 -150 -165 -180 -150]  full=-176.5
q= -176.5000000000001
CPU times: user 13.4 s, sys: 23.6 s, total: 37 s
Wall time: 40.3 s


In [ ]:
print(pd.Timestamp.now())

In [ ]:
%%time
n_jobs = 12
from multiprocessing import Pool
pool = Pool(processes=n_jobs)
y_lst = pool.map(tasks.custom_test, [x for x in [genom]*n_jobs])
pool.close()
pool.join()

In [4]:
print(1)
opt = optimize.optimizer(tasks.custom_test, genom_size=len(genom),parallel_cores=12)

1


In [ ]:
try:
    with open(root_dir+f'./genoms/best_genoms_aa_gun2.pkl', 'rb') as f:
        opt.best_genoms = pickle.load(f)
    print('loaded successfully')
except Exception:
    pass

In [5]:
opt.function(opt.best_genoms[-1], verbose=True)
for i in range(1000):
    print('opt#',i)
    opt.optimize()
    with open(root_dir+f'./genoms/best_genoms_aa_gun2.pkl', 'wb') as f:
        pickle.dump(opt.best_genoms,f,protocol=pickle.HIGHEST_PROTOCOL)
        print('WRITTEN')

aa_gun_2: [ -86.9682 -158.9841 -154.      -74.9037 -183.9931 -153.9595]  full=-174.0755235
opt# 0
scores for optimizers augmented [1.e+10 1.e+10 1.e+10 1.e+10 1.e+10 1.e+10 1.e+10 1.e+10]
chosen evol_wide previous_result: nan per tacts: nan


/home/nikita/neuroevolution-toolbox/optimize.py:36: RuntimeWarning: Mean of empty slice
  mx.append(np.nanmean(self.history_gain[opt_name])-time_penalty*np.nanmean(self.history_time[opt_name]))
/home/nikita/.local/lib/python3/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


iteration 0 y= [-65.73946687 -76.76125763 -78.31198527 -78.7660651 ]
iteration 1 y= [-65.73946687 -73.09618862 -76.76125763 -78.31198527]


In [ ]:
#draw
with open(root_dir+f'./genoms/best_genoms_aa_gun2.pkl', 'rb') as f:
    genom = pickle.load(f)
    genom = genom[-1]
print('loaded successfully')
q=tasks.custom_test(genom, verbose=True, draw=True)
print('q=',q)

In [ ]:
#save gif
tasks.video[0].save(
    root_dir+'./out_videos/best_genoms_aa_gun.gif',
    save_all=True,
    append_images=tasks.video[1:], 
    optimize=True,
    duration=200,
    loop=0
)

In [ ]:
#draw video
import cv2
writer = cv2.VideoWriter("outputvideo.mp4",cv2.VideoWriter_fourcc(*"MP4V"),30,(512,512))
for i in range(0,len(tasks.video),1):
    resize = cv2.resize(np.array(tasks.video[i]), (512, 512))
    frame = resize*0
    frame[:,:,0],frame[:,:,1],frame[:,:,2] = resize[:,:,2],resize[:,:,1], resize[:,:,0]
    writer.write(frame)
writer.release()